# Deploy Logic Apps

To operationalize our batch scoring workflow, we need a way to trigger our pipeline. Since we're applying style transfer to video data, lets trigger the pipeline everytime a new video is uploaded and detected. To do this, we'll need a mechanism that can detect the appearance of new video data. 

Logic Apps can solve this problem for us. In this notebook, we'll deploy a pre-built logic app that will look for new videos that appear in a specified storage location. When a new video is detected, the logic app will send an http request to the published pipeline (which we deployed in the previous notebook). 

---

Import the packages we need.

In [1]:
from azureml.core import Workspace, Run, Experiment
from azureml.pipeline.core import PublishedPipeline
from azureml.core.datastore import Datastore
from azure.mgmt.resource.resources.models import DeploymentMode
from azure.mgmt.resource import ResourceManagementClient
from azure.common.credentials import ServicePrincipalCredentials
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from azureml.core.authentication import AzureCliAuthentication
from pathlib import Path
import re
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

Load our workspace from the config file.

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# Also create a Project and attach to Workspace
project_folder = "scripts"
run_history_name = project_folder

if not os.path.isdir(project_folder):
    os.mkdir(project_folder)

Get authentication information about our published pipeline so that we can use it during the deployment.

In [4]:
published_pipeline = PublishedPipeline.get_published_pipeline(ws, id=get_key(env_path, "AML_PUBLISHED_PIPELINE_ID"))

In [5]:
cli_auth = AzureCliAuthentication()
aad_token = cli_auth.get_authentication_header()

### Create service principle credentials

Create service principle credentials which are used to deploy the Logic App.

In [6]:
credentials = !az ad sp create-for-rbac -o json --query "[appId, password, tenant]"

In [7]:
[sp_client, sp_secret, sp_tenant] = re.findall("\w{8}[-]\w{4}[-]\w{4}[-]\w{4}[-]\w{12}", str(credentials))

In [8]:
set_key(env_path, "SP_CLIENT", sp_client) # generated
set_key(env_path, "SP_TENANT", sp_tenant) # generated
set_key(env_path, "SP_SECRET", sp_secret) # generated

(True, 'SP_SECRET', '6a0e7f2a-cf60-4aa5-ad6c-2ff218ed0fe2')

### Deploy Logic App

![Logic Apps](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_logic_app.jpg)

The *logic* behind the Logic App deployment is shown above:
1. When a blob is added, begin the workflow.
2. Check the blob name. 
    - if the blob name ends with `.mp4`:
        - make a request to the AKS endpoint
    - otherwise:
        - terminate in cancellation


Set up credentials and resource management client to create ARM deployments.

In [10]:
# Create credentials and ARM client
credentials = ServicePrincipalCredentials(client_id=sp_client,
                                          secret=sp_secret,
                                          tenant=sp_tenant)

resource_client = ResourceManagementClient(credentials, get_key(env_path, "SUBSCRIPTION_ID"))

Create the deployment for the Azure blob storage connector.



In [11]:
# Create an Azure Blob Storage connector
with open('template.blob_connector.json') as f:
    blob_con_template = json.load(f)

blob_con_params = {
    "location": {"value": get_key(env_path, "REGION")},
    "subscription_id": {"value": get_key(env_path, "SUBSCRIPTION_ID")}
}

blob_con_props = {
    'mode': DeploymentMode.incremental,
    'template': blob_con_template,
    'parameters': blob_con_params
}

resource_client.deployments.create_or_update(
    resource_group_name=get_key(env_path, "RESOURCE_GROUP"), 
    deployment_name="blob_connector", 
    properties=blob_con_props
)

Create the deployment for the Logic App.

In [13]:
# Create Logic App
with open('template.logic_app.json') as f:
    logic_app_template = json.load(f)

logic_app_params = {
    "location": {"value": get_key(env_path, "REGION")},
    "resource_group": { "value": get_key(env_path, "RESOURCE_GROUP") },
    "name": {"value": "logic_app"},
    "subscription_id": {"value": get_key(env_path, "SUBSCRIPTION_ID")},
    "storage_container_name": {"value": get_key(env_path, "STORAGE_CONTAINER_NAME")},
    "url_endpoint": {"value": published_pipeline.endpoint},
    "aad_token": {"value": aad_token["Authorization"]},
    "datastore_name": {"value": get_key(env_path, "AML_DATASTORE_NAME")}, 
    "experiment_name": {"value": "logic_app_experiment"}
}

logic_app_props = {
    'mode': DeploymentMode.incremental,
    'template': logic_app_template,
    'parameters': logic_app_params
}

resource_client.deployments.create_or_update(
    resource_group_name=get_key(env_path, "RESOURCE_GROUP"), 
    deployment_name="logic_app", 
    properties=logic_app_props
)

Once the Logic App is deployed, go into the Azure portal and the Azure blob connector to authenticate. 

When you open up up the Azure blob connector, it should look like this:

![azure_blob_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_blob_connector_auth.PNG)

For the connector, click on the orange bar at the top to authenticate.

Once authenticated, your Logic App should be all set up and ready to trigger the workflow.

### Trigger logic app by adding a new video to the Azure blob container

In [14]:
!cp orangutan.mp4 trigger_test_orangutan.mp4

In [15]:
my_datastore = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name=get_key(env_path, "AML_DATASTORE_NAME"), 
    container_name=get_key(env_path, "STORAGE_CONTAINER_NAME"), 
    account_name=get_key(env_path, "STORAGE_ACCOUNT_NAME"), 
    account_key=get_key(env_path, "STORAGE_ACCOUNT_KEY"),
    overwrite=True
)

In [16]:
# Upload new trigger file video
my_datastore.upload_files(
    ["./trigger_test_orangutan.mp4"],
    overwrite=True
)

$AZUREML_DATAREFERENCE_datastore

The appearance of the new `trigger_test_orangutan.mp4` video will trigger the Logic App flow. Inspect your logic app in the portal to see the progress.

---

You are now ready to move on to the [next notebook](05_clean_up.ipynb).